[The code is from this Github repo of mine](https://github.com/ceshine/finetuning-t5/tree/32704724dae697121db905eda813bdb74f78c480/mnli). The inference script is written to `tmp.py` to avoid the version conflict error from the transformers libary.

The mT5-base model was [trained on the MNLI dataset (Enlgish-only)](https://www.kaggle.com/ceshine/preprocess-and-finetune-mt5?scriptVersionId=52689872). Therefore this is zero-shot learning for all the non-English sentences in the dataset.

In [ ]:
!pip install --upgrade pip
!pip uninstall -y allennlp
!pip install transformers==4.1.1 typer
!pip install https://github.com/veritable-tech/pytorch-lightning-spells/archive/master.zip

In [ ]:
!ls /kaggle/input/mt5-base-mnli-pretrained

In [ ]:
%%writefile tmp.py
#!/usr/bin/env python
import contextlib as __stickytape_contextlib

@__stickytape_contextlib.contextmanager
def __stickytape_temporary_dir():
    import tempfile
    import shutil
    dir_path = tempfile.mkdtemp()
    try:
        yield dir_path
    finally:
        shutil.rmtree(dir_path)

with __stickytape_temporary_dir() as __stickytape_working_dir:
    def __stickytape_write_module(path, contents):
        import os, os.path

        def make_package(path):
            parts = path.split("/")
            partial_path = __stickytape_working_dir
            for part in parts:
                partial_path = os.path.join(partial_path, part)
                if not os.path.exists(partial_path):
                    os.mkdir(partial_path)
                    with open(os.path.join(partial_path, "__init__.py"), "w") as f:
                        f.write("\n")

        make_package(os.path.dirname(path))

        full_path = os.path.join(__stickytape_working_dir, path)
        with open(full_path, "w") as module_file:
            module_file.write(contents)

    import sys as __stickytape_sys
    __stickytape_sys.path.insert(0, __stickytape_working_dir)

    __stickytape_write_module('preprocess/tokenize_dataset.py', 'import enum\nfrom pathlib import Path\n\nimport typer\nimport joblib\nimport numpy as np\nimport pandas as pd\nfrom tqdm import tqdm\nfrom transformers import MT5Tokenizer, T5Tokenizer\n\nDATA_PATH = Path("data/")\nCACHE_PATH = Path("cache/")\nCACHE_PATH.mkdir(exist_ok=True, parents=True)\n\n# label_dict = {\n#     0: "entailment",\n#     1: "neutral",\n#     2: "contradiction"\n# }\n\n\nclass Corpus(enum.Enum):\n    KAGGLE = "kaggle"\n    MNLI = "multinli"\n\n\ndef process_file(data: pd.DataFrame, tokenizer: MT5Tokenizer, batch_size: int):\n    premise_ids = []\n    hypothesis_ids = []\n    if "label" in data.columns:\n        assert data.label.max() == data.label.nunique() - 1\n        assert data.label.min() == 0\n        labels = data.label.astype("int").values[:, np.newaxis]\n    else:\n        labels = None\n    for i in tqdm(range(0, len(data), batch_size), ncols=100):\n        batch = data.iloc[i:i+batch_size]\n        # tokenize premise\n        premise_ids.extend(\n            tokenizer.batch_encode_plus(\n                batch["premise"].values, return_attention_mask=False\n            )["input_ids"]\n        )\n        # tokenize hypothesis\n        hypothesis_ids.extend(\n            tokenizer.batch_encode_plus(\n                batch["hypothesis"].values.tolist(), return_attention_mask=False\n            )["input_ids"]\n        )\n    return premise_ids, hypothesis_ids, labels\n\n\ndef main(dataset: Corpus, tokenizer_name: str = "google/mt5-small", batch_size: int = 1024):\n    (CACHE_PATH / dataset.value).mkdir(parents=True, exist_ok=True)\n    if "mt5" in tokenizer_name:\n        tokenizer = MT5Tokenizer.from_pretrained(tokenizer_name)\n    else:\n        tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)\n    for datafile in ("train_split.csv", "valid.csv", "test.csv", "test_matched.csv", "test_mismatched.csv"):\n        if not (DATA_PATH / dataset.value / datafile).exists():\n            continue\n        print(datafile)\n        data = pd.read_csv(DATA_PATH / dataset.value / datafile)\n        premise_ids, hypothesis_ids, labels = process_file(data, tokenizer, batch_size)\n        joblib.dump([premise_ids, hypothesis_ids, labels], CACHE_PATH / dataset.value / (Path(datafile).stem + ".jbl"))\n        print(CACHE_PATH / dataset.value / (Path(datafile).stem + ".jbl"))\n\n\nif __name__ == "__main__":\n    typer.run(main)\n')
    __stickytape_write_module('preprocess/__init__.py', '')
    __stickytape_write_module('train.py', 'import os\nimport gc\nimport math\nimport json\nimport shutil\nfrom itertools import chain\nfrom pathlib import Path\nfrom typing import List, Optional\nfrom dataclasses import dataclass, asdict\n\nimport typer\nimport torch\nimport joblib\nimport numpy as np\nfrom torch.utils.data import Dataset\nimport pytorch_lightning as pl\nimport pytorch_lightning_spells as pls\nfrom torch.optim.lr_scheduler import CosineAnnealingLR\nfrom transformers import (\n    MT5ForConditionalGeneration, MT5Tokenizer, T5ForConditionalGeneration, T5Tokenizer,\n    MT5Config, T5Config  # , Adafactor\n)\n\nfrom t2t import BaseConfig, T5BaseModel, single_token_cross_entropy_loss\nfrom preprocess.tokenize_dataset import Corpus\n\nCACHE_DIR = Path("cache/")\nCACHE_DIR.mkdir(exist_ok=True, parents=True)\n\n\nclass Adafactor(torch.optim.Optimizer):\n    """\n    AdaFactor pytorch implementation can be used as a drop in replacement for Adam original fairseq code:\n    https://github.com/pytorch/fairseq/blob/master/fairseq/optim/adafactor.py\n\n    Paper: `Adafactor: Adaptive Learning Rates with Sublinear Memory Cost` https://arxiv.org/abs/1804.04235 Note that\n    this optimizer internally adjusts the learning rate depending on the *scale_parameter*, *relative_step* and\n    *warmup_init* options. To use a manual (external) learning rate schedule you should set `scale_parameter=False` and\n    `relative_step=False`.\n\n    Arguments:\n        params (:obj:`Iterable[torch.nn.parameter.Parameter]`):\n            Iterable of parameters to optimize or dictionaries defining parameter groups.\n        lr (:obj:`float`, `optional`):\n            The external learning rate.\n        eps (:obj:`Tuple[float, float]`, `optional`, defaults to (1e-30, 1e-3)):\n            Regularization constants for square gradient and parameter scale respectively\n        clip_threshold (:obj:`float`, `optional`, defaults 1.0):\n            Threshold of root mean square of final gradient update\n        decay_rate (:obj:`float`, `optional`, defaults to -0.8):\n            Coefficient used to compute running averages of square\n        beta1 (:obj:`float`, `optional`):\n            Coefficient used for computing running averages of gradient\n        weight_decay (:obj:`float`, `optional`, defaults to 0):\n            Weight decay (L2 penalty)\n        scale_parameter (:obj:`bool`, `optional`, defaults to :obj:`True`):\n            If True, learning rate is scaled by root mean square\n        relative_step (:obj:`bool`, `optional`, defaults to :obj:`True`):\n            If True, time-dependent learning rate is computed instead of external learning rate\n        warmup_init (:obj:`bool`, `optional`, defaults to :obj:`False`):\n            Time-dependent learning rate computation depends on whether warm-up initialization is being used\n\n    This implementation handles low-precision (FP16, bfloat) values, but we have not thoroughly tested.\n\n    Recommended T5 finetuning settings:\n\n        - Scheduled LR warm-up to fixed LR\n        - disable relative updates\n        - use clip threshold: https://arxiv.org/abs/2004.14546\n\n        Example::\n\n            Adafactor(model.parameters(), lr=1e-3, relative_step=False, warmup_init=True)\n\n        - Alternatively, relative_step with warmup_init can be used.\n        - Training without LR warmup or clip threshold is not recommended. Additional optimizer operations like\n          gradient clipping should not be used alongside Adafactor.\n\n    Usage::\n\n        # replace AdamW with Adafactor\n        optimizer = Adafactor(\n            model.parameters(),\n            lr=1e-3,\n            eps=(1e-30, 1e-3),\n            clip_threshold=1.0,\n            decay_rate=-0.8,\n            beta1=None,\n            weight_decay=0.0,\n            relative_step=False,\n            scale_parameter=False,\n            warmup_init=False\n        )\n    """\n\n    def __init__(\n        self,\n        params,\n        lr=None,\n        eps=(1e-30, 1e-3),\n        clip_threshold=1.0,\n        decay_rate=-0.8,\n        beta1=None,\n        weight_decay=0.0,\n        scale_parameter=True,\n        relative_step=True,\n        warmup_init=False,\n    ):\n        if lr is not None and relative_step:\n            raise ValueError("Cannot combine manual lr and relative_step options")\n        if warmup_init and not relative_step:\n            raise ValueError("warmup_init requires relative_step=True")\n\n        defaults = dict(\n            lr=lr,\n            eps=eps,\n            clip_threshold=clip_threshold,\n            decay_rate=decay_rate,\n            beta1=beta1,\n            weight_decay=weight_decay,\n            scale_parameter=scale_parameter,\n            relative_step=relative_step,\n            warmup_init=warmup_init,\n        )\n        super().__init__(params, defaults)\n\n    @staticmethod\n    def _get_lr(param_group, param_state):\n        rel_step_sz = param_group["lr"]\n        if param_group["relative_step"]:\n            min_step = 1e-6 * param_state["step"] if param_group["warmup_init"] else 1e-2\n            rel_step_sz = min(min_step, 1.0 / math.sqrt(param_state["step"]))\n        param_scale = 1.0\n        if param_group["scale_parameter"]:\n            param_scale = max(param_group["eps"][1], param_state["RMS"])\n        return param_scale * rel_step_sz\n\n    @staticmethod\n    def _get_options(param_group, param_shape):\n        factored = len(param_shape) >= 2\n        use_first_moment = param_group["beta1"] is not None\n        return factored, use_first_moment\n\n    @staticmethod\n    def _rms(tensor):\n        return tensor.norm(2) / (tensor.numel() ** 0.5)\n\n    @staticmethod\n    def _approx_sq_grad(exp_avg_sq_row, exp_avg_sq_col):\n        r_factor = (exp_avg_sq_row / exp_avg_sq_row.mean(dim=-1, keepdim=True)).rsqrt_()\n        c_factor = exp_avg_sq_col.rsqrt()\n        return torch.mm(r_factor.unsqueeze(-1), c_factor.unsqueeze(0))\n\n    def step(self, closure=None):\n        """\n        Performs a single optimization step\n\n        Arguments:\n            closure (callable, optional): A closure that reevaluates the model\n                and returns the loss.\n        """\n        loss = None\n        if closure is not None:\n            loss = closure()\n\n        for group in self.param_groups:\n            for i, p in enumerate(group["params"]):\n                if p.grad is None:\n                    continue\n                grad = p.grad.data\n                if grad.dtype in {torch.float16, torch.bfloat16}:\n                    grad = grad.float()\n                if grad.is_sparse:\n                    raise RuntimeError("Adafactor does not support sparse gradients.")\n\n                state = self.state[p]\n                grad_shape = grad.shape\n\n                factored, use_first_moment = self._get_options(group, grad_shape)\n                # State Initialization\n                if len(state) == 0:\n                    state["step"] = 0\n\n                    if use_first_moment:\n                        # Exponential moving average of gradient values\n                        state["exp_avg"] = torch.zeros_like(grad)\n                    if factored:\n                        state["exp_avg_sq_row"] = torch.zeros(grad_shape[:-1]).to(grad)\n                        state["exp_avg_sq_col"] = torch.zeros(grad_shape[:-2] + grad_shape[-1:]).to(grad)\n                    else:\n                        state["exp_avg_sq"] = torch.zeros_like(grad)\n\n                    state["RMS"] = 0\n                else:\n                    if use_first_moment:\n                        state["exp_avg"] = state["exp_avg"].to(grad)\n                    if factored:\n                        state["exp_avg_sq_row"] = state["exp_avg_sq_row"].to(grad)\n                        state["exp_avg_sq_col"] = state["exp_avg_sq_col"].to(grad)\n                    else:\n                        state["exp_avg_sq"] = state["exp_avg_sq"].to(grad)\n\n                p_data_fp32 = p.data\n                if p.data.dtype in {torch.float16, torch.bfloat16}:\n                    p_data_fp32 = p_data_fp32.float()\n\n                state["step"] += 1\n                state["RMS"] = self._rms(p_data_fp32)\n                lr = self._get_lr(group, state)\n\n                beta2t = 1.0 - math.pow(state["step"], group["decay_rate"])\n                update = (grad ** 2) + group["eps"][0]\n                if factored:\n                    exp_avg_sq_row = state["exp_avg_sq_row"]\n                    exp_avg_sq_col = state["exp_avg_sq_col"]\n\n                    exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))\n                    exp_avg_sq_col.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-2))\n\n                    # Approximation of exponential moving average of square of gradient\n                    update = self._approx_sq_grad(exp_avg_sq_row, exp_avg_sq_col)\n                    update.mul_(grad)\n                else:\n                    exp_avg_sq = state["exp_avg_sq"]\n\n                    exp_avg_sq.mul_(beta2t).add_(1.0 - beta2t, update)\n                    update = exp_avg_sq.rsqrt().mul_(grad)\n\n                update.div_((self._rms(update) / group["clip_threshold"]).clamp_(min=1.0))\n                update.mul_(lr)\n\n                if use_first_moment:\n                    exp_avg = state["exp_avg"]\n                    exp_avg.mul_(group["beta1"]).add_(1 - group["beta1"], update)\n                    update = exp_avg\n\n                if group["weight_decay"] != 0:\n                    p_data_fp32.add_(-group["weight_decay"] * lr, p_data_fp32)\n\n                p_data_fp32.add_(-update)\n\n                if p.data.dtype in {torch.float16, torch.bfloat16}:\n                    p.data.copy_(p_data_fp32)\n\n        return loss\n\n\n@dataclass\nclass Config(BaseConfig):\n    dataset: Corpus = Corpus.KAGGLE\n    decoder_only: bool = True\n    freeze_embeddings: bool = False\n    num_classes: int = 3\n\n\ndef shrink_vocab(model_path, model):\n    if (Path(model_path) / "kept_ids.json").exists():\n        with open(Path(model_path) / "kept_ids.json") as fin:\n            kept_ids = np.asarray(json.load(fin))\n        extracted_weights = model.shared.weight.data[kept_ids]\n        print(extracted_weights.shape)\n        print(model.shared.weight.shape)\n        model.shared = torch.nn.Embedding(*extracted_weights.shape)\n        model.shared.weight.data = extracted_weights\n        model.decoder.embed_tokens = model.shared\n        model.encoder.embed_tokens = model.shared\n\n\ndef load_model(model_class, model_config_class, config):\n    model_path = config.base_t5_model\n    try:\n        model = model_class.from_pretrained(model_path)\n        # replace the lm_head\n        model.lm_head = torch.nn.Linear(model.lm_head.in_features, config.num_classes, bias=False)\n        shrink_vocab(model_path, model)\n    except RuntimeError:\n        model = model_class(\n            model_config_class.from_pretrained(model_path)\n        )\n        model.lm_head = torch.nn.Linear(model.lm_head.in_features, config.num_classes, bias=False)\n        shrink_vocab(model_path, model)\n        model.load_state_dict(torch.load(str(Path(model_path) / "pytorch_model.bin")))\n    return model\n\n\nclass T5Model(T5BaseModel):\n    def __init__(self, config: Config, **kwargs):\n        if "mt5" in config.base_t5_model:\n            tokenizer = MT5Tokenizer.from_pretrained(config.base_t5_model)\n            model = load_model(MT5ForConditionalGeneration, MT5Config, config)\n            # tie the weights\n            # model.lm_head.weight = model.shared.weight\n        else:\n            tokenizer = T5Tokenizer.from_pretrained(config.base_t5_model)\n            model = load_model(T5ForConditionalGeneration, T5Config, config)\n        super().__init__(config, model, tokenizer, is_classifier=True)\n        self.config = config\n        # log the config values\n        self.save_hyperparameters(asdict(config))\n        self.context_tokens_1 = self.tokenizer.encode("xnli hypothesis:")[:-1]\n        self.context_tokens_2 = self.tokenizer.encode("premise:")[:-1]\n        self.train_dataset = XNLIDataset(\n            self.config.dataset, \'train_split.jbl\',\n            self.context_tokens_1, self.context_tokens_2,\n            max_len=config.max_len // 2\n        )  # , tokenizer)\n        print("Train dataset: ", len(self.train_dataset))\n        self.valid_dataset = XNLIDataset(\n            self.config.dataset, \'valid.jbl\', self.context_tokens_1, self.context_tokens_2,\n            max_len=config.max_len // 2\n        )\n        print("Valid dataset: ", len(self.valid_dataset))\n\n    def configure_optimizers(self):\n        if self.config.decoder_only:\n            print("Decoder only")\n            pls.utils.set_trainable(self.model.encoder.block, False)\n            pls.utils.set_trainable(self.model.encoder.final_layer_norm, False)\n            params = self.model.lm_head.parameters()\n            if not (self.model.lm_head.weight is self.model.shared.weight):\n                # weight is not shared\n                pls.utils.set_trainable(self.model.shared, False)\n                params = chain(\n                    self.model.decoder.block.parameters(),\n                    self.model.decoder.final_layer_norm.parameters(),\n                    self.model.lm_head.parameters()\n                )\n            else:\n                pls.utils.set_trainable(self.model.decoder.block, False)\n                pls.utils.set_trainable(self.model.decoder.final_layer_norm, False)\n                params = self.model.lm_head.parameters()\n            optimizer = Adafactor(  # torch.optim.AdamW(\n                [\n                    {\n                        "params": params,\n                        # "learning_rate": self.config.learning_rate,\n                        # "weight_decay": self.config.weight_decay\n\n                    }\n                ],\n                relative_step=False, warmup_init=False,\n                clip_threshold=1.0, lr=self.config.learning_rate,\n                scale_parameter=True\n            )\n        elif self.config.freeze_embeddings and not (self.model.lm_head.weight is self.model.shared.weight):\n            print("Embeddings frozen")\n            # freezing embeddings is pointless when the weights are tied\n            pls.utils.set_trainable(self.model.shared, False)\n            optimizer = Adafactor(\n                chain(\n                    self.model.encoder.block.parameters(),\n                    self.model.encoder.final_layer_norm.parameters(),\n                    self.model.decoder.block.parameters(),\n                    self.model.decoder.final_layer_norm.parameters(),\n                    self.model.lm_head.parameters()\n                ),\n                relative_step=False, warmup_init=False,\n                clip_threshold=1.0, lr=self.config.learning_rate,\n                scale_parameter=True\n            )\n        else:\n            print("Full model")\n            # # make sure the weights are tied\n            # assert self.model.lm_head.weight is self.model.shared.weight, (\n            #     self.model.shared.weight - self.model.lm_head.weight).sum()\n            optimizer = Adafactor(\n                self.model.parameters(),\n                relative_step=False, warmup_init=False,\n                clip_threshold=1.0, lr=self.config.learning_rate,\n                scale_parameter=True\n            )\n            #     [\n            #         {\n            #             "params": (\n            #                 chain(\n            #                     self.model.encoder.block.parameters(),\n            #                     self.model.encoder.final_layer_norm.parameters()\n            #                 ) if (self.model.lm_head.weight is self.model.shared.weight) else\n            #                 self.model.encoder.parameters()\n            #             ),\n            #             "learning_rate": self.config.learning_rate / 2,\n            #             "weight_decay": self.config.weight_decay / 2,\n            #         },\n            #         {\n            #             "params": chain(\n            #                 self.model.decoder.block.parameters(),\n            #                 self.model.decoder.final_layer_norm.parameters(),\n            #                 self.model.lm_head.parameters()\n            #             ),\n            #             "learning_rate": self.config.learning_rate,\n            #             "weight_decay": self.config.weight_decay\n\n            #         }\n            #     ]\n            # )\n        print("Optimizer parameter count: {:,d}".format(np.sum([\n            pls.utils.count_parameters(group["params"]) for group in optimizer.param_groups\n        ])))\n        steps_per_epochs = math.floor(\n            len(self.train_dataset) / self.config.batch_size / self.config.grad_accu  # / self.num_gpus # dpp mode\n        )\n        print("Steps per epochs:", steps_per_epochs)\n        n_steps = steps_per_epochs * self.config.epochs\n        lr_durations = [\n            int(n_steps*0.1),\n            int(np.ceil(n_steps*0.9)) + 1\n        ]\n        break_points = [0] + list(np.cumsum(lr_durations))[:-1]\n        scheduler = {\n            \'scheduler\': pls.lr_schedulers.MultiStageScheduler(\n                [\n                    pls.lr_schedulers.LinearLR(optimizer, 0.0001, lr_durations[0]),\n                    CosineAnnealingLR(optimizer, lr_durations[1])\n                ],\n                start_at_epochs=break_points\n            ),\n            \'interval\': \'step\',\n            \'frequency\': 1,\n            \'strict\': True,\n        }\n        return {\n            \'optimizer\': optimizer,\n            \'lr_scheduler\': scheduler\n        }\n\n\nclass XNLIDataset(Dataset):\n    def __init__(\n            self, corpus: Corpus, file_name: str, context_tokens_1: List[int], context_tokens_2: List[int],\n            max_len: int = 128, tokenizer=None):\n        self.premise_ids, self.hypothesis_ids, self.labels = joblib.load(\n            CACHE_DIR / corpus.value / f\'{file_name}\')\n        self.context_tokens_1 = context_tokens_1\n        self.context_tokens_2 = context_tokens_2\n        self.max_len = max_len  # max length for premise and hypothesis\n        # for debug\n        self.tokenizer = tokenizer\n\n    def __len__(self):\n        return len(self.premise_ids)\n\n    def __getitem__(self, index):\n        if self.labels is None:\n            label = torch.tensor([0], dtype=torch.int64)\n        else:\n            label = torch.tensor(self.labels[index], dtype=torch.int64)\n        premise_length = min(self.max_len, len(self.premise_ids[index]))\n        hypothesis_length = min(self.max_len-1, len(self.hypothesis_ids[index]) - 1)\n        if self.tokenizer:\n            print(self.tokenizer.decode(self.context_tokens_1 + self.hypothesis_ids[index][:hypothesis_length] +\n                                        self.context_tokens_2 + self.premise_ids[index][:premise_length]))\n            print(self.tokenizer.decode(self.labels[index]))\n        return (\n            torch.tensor(\n                self.context_tokens_1 + self.hypothesis_ids[index][:-1][:hypothesis_length] +\n                self.context_tokens_2 + self.premise_ids[index][:premise_length],\n                dtype=torch.int64\n            ),\n            label\n        )\n\n\ndef main(\n    t5_model: str = "google/mt5-small", lr: float = 1e-4,\n    epochs: int = 5, fp16: bool = False,\n    dataset: Corpus = "kaggle", batch_size: int = 16,\n    max_len: int = 64, grad_accu: int = 1,\n    num_gpus: int = 1, disable_progress_bar: bool = False,\n    valid_frequency: Optional[float] = None,\n    full_model: bool = False, tpu_cores: int = 0,\n    freeze_embeddings: bool = False\n):\n    pl.seed_everything(int(os.environ.get("SEED", 738)))\n    config = Config(\n        base_t5_model=t5_model,\n        learning_rate=lr,\n        epochs=epochs,\n        dataset=dataset,\n        max_len=max_len,\n        grad_accu=grad_accu,\n        batch_size=batch_size,\n        fp16=fp16,\n        weight_decay=0,\n        num_gpus=num_gpus if tpu_cores == 0 else 0,\n        tpu_cores=tpu_cores,\n        loss_fn=single_token_cross_entropy_loss,\n        decoder_only=not (full_model or freeze_embeddings),\n        freeze_embeddings=freeze_embeddings\n    )\n    # print(config)\n\n    pl_module = T5Model(config)\n\n    callbacks = [\n        pl.callbacks.ModelCheckpoint(\n            dirpath=str(CACHE_DIR / "model_checkpoints"),\n            monitor=\'val_loss\',\n            mode="min",\n            filename=\'{step:06d}-{val_loss:.4f}\',\n            save_top_k=1,\n            save_last=False\n        ),\n        pl.callbacks.LearningRateMonitor(logging_interval=\'step\'),\n    ]\n    trainer = pl.Trainer(\n        accelerator=\'dp\' if num_gpus > 1 else None,\n        # amp_backend="apex", amp_level=\'O1\',\n        precision=16 if config.fp16 else 32,\n        gpus=config.num_gpus if tpu_cores == 0 else None,\n        val_check_interval=valid_frequency if valid_frequency else 1.0,\n        # gradient_clip_val=3,\n        max_epochs=epochs,\n        # max_steps=steps,\n        callbacks=callbacks,\n        accumulate_grad_batches=grad_accu,\n        # auto_scale_batch_size=\'power\' if batch_size is None else None,\n        logger=[\n            pl.loggers.TensorBoardLogger(str(CACHE_DIR / "tb_logs"), name=""),\n            pls.loggers.ScreenLogger(),\n            # pl.loggers.WandbLogger(project="t5-paraphrase")\n        ],\n        log_every_n_steps=100,\n        progress_bar_refresh_rate=0 if disable_progress_bar else (20 if tpu_cores == 0 else 5),\n        tpu_cores=config.tpu_cores if config.tpu_cores else None\n    )\n\n    trainer.fit(pl_module)\n\n    model_name = config.base_t5_model.split("/")[-1]\n\n    assert isinstance(callbacks[0], pl.callbacks.ModelCheckpoint)\n    pl_module.load_state_dict(torch.load(callbacks[0].best_model_path)["state_dict"])\n    del trainer\n    gc.collect()\n    save_path = CACHE_DIR / f"{model_name}_best"\n    pl_module.model.save_pretrained(save_path)\n    pl_module.tokenizer.save_pretrained(save_path)\n    if (Path(t5_model) / "kept_ids.json").exists():\n        shutil.copy(Path(t5_model) / "kept_ids.json", save_path / "kept_ids.json")\n    print("Best model saved")\n\n\nif __name__ == "__main__":\n    typer.run(main)\n')
    __stickytape_write_module('t2t/__init__.py', 'import math\nfrom functools import partial\nfrom dataclasses import dataclass\nfrom typing import Callable, Union\n\nimport torch\nimport torch.nn.functional as F\nfrom torch.utils.data.sampler import Sampler\nimport numpy as np\nimport pytorch_lightning as pl\nimport pytorch_lightning_spells as pls\nfrom torch.utils.data import DataLoader, Dataset\nfrom torch.optim.lr_scheduler import CosineAnnealingLR\nfrom transformers import (\n    T5ForConditionalGeneration, T5Tokenizer, MT5ForConditionalGeneration, MT5Tokenizer\n)\n\n\ndef masked_cross_entropy_loss(outputs, targets):\n    # print(targets["ids"].shape, outputs.shape)\n    targets, mask = targets["ids"], targets["mask"]\n    loss = torch.sum(\n        mask.view(-1) * F.cross_entropy(\n            outputs.view(-1, outputs.size(2)),\n            targets.view(-1),\n            reduction="none"\n        )\n    ) / mask.sum()\n    return loss\n\n\ndef single_token_cross_entropy_loss(outputs, targets):\n    target_ids = targets["ids"]\n    # print(outputs[:, :, 333])\n    loss = F.cross_entropy(\n        outputs.view(-1, outputs.size(2)),\n        target_ids.view(-1)\n    )\n    return loss\n\n\ndef optimize_sequence(ids, pad, max_len):\n    # Pad to the minimum multiple of 8 to utilize tensor cores\n    batch_max_len = np.max([x.size(0) for x in ids])\n    if batch_max_len > 8:\n        max_length = math.ceil(\n            min(max_len, batch_max_len) / 8.\n        ) * 8\n    else:\n        max_length = batch_max_len\n    padded_ids = ids[0].new_zeros((len(ids), max_length)) + pad\n    mask = ids[0].new_zeros((len(ids), max_length))\n    for i, example in enumerate(ids):\n        example = example[:max_len]\n        padded_ids[i, :len(example)] = example\n        mask[i, :len(example)] = 1\n    return padded_ids, mask\n\n\ndef collate_batch(batch, max_len, pad=0, decode_start_token=0, is_classifier=False):\n    """Batch preparation.\n\n    Truncate the sequence to reduce wastes.\n    """\n    source_ids, target_ids = zip(*batch)\n    source_ids, src_mask = optimize_sequence(source_ids, pad, max_len)\n    if not is_classifier:\n        target_ids, target_mask = optimize_sequence(target_ids, pad, max_len)\n        shifted_target_ids = target_ids.new_zeros(target_ids.shape)\n        shifted_target_ids[..., 1:] = target_ids[..., :-1].clone()\n        shifted_target_ids[..., 0] = decode_start_token\n        return (\n            {\n                "input_ids": source_ids, "attention_mask": src_mask,\n                "decoder_input_ids": shifted_target_ids\n            },\n            {"ids": target_ids, "mask": target_mask}\n        )\n    # is classifier\n    target_ids = torch.stack(target_ids)\n    shifted_target_ids = target_ids.new_zeros(target_ids.shape[0], 1) + decode_start_token\n    return (\n        {\n            "input_ids": source_ids, "attention_mask": src_mask,\n            "decoder_input_ids": shifted_target_ids\n        },\n        {"ids": target_ids}\n    )\n\n\nclass SortSampler(Sampler):\n    def __init__(self, data_source, key):\n        self.data_source, self.key = data_source, key\n\n    def __len__(self):\n        return len(self.data_source)\n\n    def __iter__(self):\n        return iter(sorted(range(len(self.data_source)), key=self.key, reverse=True))\n\n\nclass SortishSampler(Sampler):\n    """Returns an iterator that traverses the the data in randomly ordered batches that are approximately the same size.\n    The max key size batch is always returned in the first call because of pytorch cuda memory allocation sequencing.\n    Without that max key returned first multiple buffers may be allocated when the first created isn\'t large enough\n    to hold the next in the sequence.\n\n    Taken from fastai library.\n    """\n\n    def __init__(self, data_source, key, bs, chunk_size=50):\n        self.data_source, self.key, self.bs = data_source, key, bs\n        self.chunk_size = 50\n\n    def __len__(self): return len(self.data_source)\n\n    def __iter__(self):\n        while True:\n            idxs = np.random.permutation(len(self.data_source))\n            sz = self.bs * self.chunk_size\n            ck_idx = [idxs[i:i+sz] for i in range(0, len(idxs), sz)]\n            sort_idx = np.concatenate(\n                [sorted(s, key=self.key, reverse=True) for s in ck_idx])\n            sz = self.bs\n            ck_idx = [sort_idx[i:i+sz]for i in range(0, len(sort_idx), sz)]\n            # find the chunk with the largest key,\n            max_ck = np.argmax([self.key(ck[0]) for ck in ck_idx])\n            # then make sure it goes first.\n            if len(ck_idx[max_ck]) != self.bs:\n                continue\n            ck_idx[0], ck_idx[max_ck] = ck_idx[max_ck], ck_idx[0]\n            sort_idx = np.concatenate(np.random.permutation([\n                np.random.permutation(chunk.reshape(self.bs, -1)).reshape(-1)\n                for chunk in ck_idx[1:-1]\n            ]))\n            sort_idx = np.concatenate((ck_idx[0], sort_idx, ck_idx[-1]))\n            break\n        return iter(sort_idx)\n\n\n@dataclass\nclass BaseConfig:\n    base_t5_model: str\n    batch_size: int\n    fp16: bool\n    learning_rate: float\n    weight_decay: float\n    epochs: int\n    max_len: int\n    loss_fn: Callable\n    num_gpus: int = 1\n    grad_accu: int = 1\n    tpu_cores: int = 0\n\n\nclass T5BaseModel(pl.LightningModule):\n    def __init__(\n            self, config: BaseConfig, model: Union[T5ForConditionalGeneration, MT5ForConditionalGeneration],\n            tokenizer: Union[T5Tokenizer, MT5Tokenizer], is_classifier: bool = False, **kwargs):\n        super().__init__()\n        self.config = config\n        # placeholder for pylint\n        self.train_dataset: Dataset = Dataset()\n        self.valid_dataset: Dataset = Dataset()\n        # the actual stuffs\n        self.model = model\n        self.tokenizer = tokenizer\n        self.collate_fn = partial(\n            collate_batch, pad=self.model.config.decoder_start_token_id,\n            decode_start_token=self.model.config.pad_token_id,\n            max_len=self.config.max_len, is_classifier=is_classifier\n        )\n        self.metrics = [\n            ("acc", pl.metrics.Accuracy(compute_on_step=False))\n        ]\n        self.train_loss_tracker = pls.utils.EMATracker(alpha=0.02)\n\n    def forward(self, input_tensors):\n        return self.model(**input_tensors)[0]\n\n    def train_dataloader(self):\n        sampler = SortishSampler(\n            self.train_dataset,\n            key=lambda x: len(self.train_dataset[x][0]),\n            bs=self.config.batch_size\n        )\n        return DataLoader(\n            self.train_dataset, num_workers=4 if self.config.tpu_cores == 0 else 1, shuffle=False, drop_last=True,\n            batch_size=self.config.batch_size, collate_fn=self.collate_fn, sampler=sampler)\n\n    def get_progress_bar_dict(self):\n        # don\'t show the experiment version number\n        items = super().get_progress_bar_dict()\n        items.pop("v_num", None)\n        return items\n\n    def val_dataloader(self):\n        return DataLoader(\n            self.valid_dataset, num_workers=4 if self.config.tpu_cores == 0 else 1, shuffle=False, drop_last=False,\n            batch_size=self.config.batch_size*2, collate_fn=self.collate_fn)\n\n    def validation_step(self, batch, batch_idx):\n        logits = self.forward(batch[0])\n        loss = self.config.loss_fn(\n            logits,\n            batch[1]\n        )\n        preds = torch.argmax(logits, dim=-1)[:, :batch[1]["ids"].size(1)]\n        return {\n            \'loss\': loss,\n            \'preds\': preds,\n            \'target\': batch[1]\n        }\n\n    def validation_step_end(self, outputs):\n        self.log(\'val_loss\', outputs[\'loss\'].mean())\n        for name, metric in self.metrics:\n            metric(\n                outputs[\'preds\'].view(-1).cpu(),\n                outputs[\'target\'][\'ids\'].view(-1).cpu()\n            )\n            self.log("val_" + name, metric)\n\n    def _should_log(self, flag):\n        if (self.trainer.global_step + 1) % self.trainer.log_every_n_steps == 0:\n            if isinstance(flag, list):\n                return flag[0]\n            return flag\n        return False\n\n    def training_step_end(self, outputs):\n        loss = outputs["loss"].mean()\n        self.train_loss_tracker.update(loss.detach())\n        if self._should_log(outputs["log"]):\n            self.logger.log_metrics({\n                "train_loss": self.train_loss_tracker.value\n            }, step=self.global_step)\n        return loss\n\n    def training_step(self, batch, batch_idx):\n        loss = self.config.loss_fn(\n            self.forward(batch[0]),\n            batch[1]\n        )\n        return {"loss": loss, "log": batch_idx % self.trainer.accumulate_grad_batches == 0}\n\n    def configure_optimizers(self):\n        optimizer = pls.optimizers.RAdam(\n            self.model.parameters(), lr=self.config.learning_rate,\n            weight_decay=self.config.weight_decay\n        )\n        steps_per_epochs = math.floor(\n            len(self.train_dataset) / self.config.batch_size / self.config.grad_accu  # / self.num_gpus # dpp mode\n        )\n        print("Steps per epochs:", steps_per_epochs)\n        n_steps = steps_per_epochs * self.config.epochs\n        lr_durations = [\n            int(n_steps*0.05),\n            int(np.ceil(n_steps*0.95)) + 1\n        ]\n        break_points = [0] + list(np.cumsum(lr_durations))[:-1]\n        scheduler = {\n            \'scheduler\': pls.lr_schedulers.MultiStageScheduler(\n                [\n                    pls.lr_schedulers.LinearLR(optimizer, 0.01, lr_durations[0]),\n                    CosineAnnealingLR(optimizer, lr_durations[1])\n                ],\n                start_at_epochs=break_points\n            ),\n            \'interval\': \'step\',\n            \'frequency\': 1,\n            \'strict\': True,\n        }\n        return {\n            \'optimizer\': optimizer,\n            \'lr_scheduler\': scheduler\n        }\n')
    from pathlib import Path
    from functools import partial
    
    import torch
    import numpy as np
    import pandas as pd
    from transformers import MT5ForConditionalGeneration, MT5Tokenizer, MT5Config
    from torch.utils.data import DataLoader
    
    from preprocess.tokenize_dataset import process_file
    from train import XNLIDataset, shrink_vocab
    from t2t import collate_batch
    
    df_test = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
    
    MODEL_PATH = "/kaggle/input/mt5-base-mnli-pretrained/mt5-base_best"
    tokenizer = MT5Tokenizer.from_pretrained(MODEL_PATH)
    
    model = MT5ForConditionalGeneration(
        MT5Config.from_pretrained(MODEL_PATH)
    )
    model.lm_head = torch.nn.Linear(model.lm_head.in_features, 3, bias=False)
    shrink_vocab(MODEL_PATH, model)
    model.load_state_dict(torch.load(str(Path(MODEL_PATH) / "pytorch_model.bin")))
    model = model.cuda().eval()
    
    label_tokens_dict = {
        tokens[0]: idx for idx, tokens in enumerate(tokenizer.batch_encode_plus(
            ["entailment", "neutral", "contradiction"]
        )["input_ids"])
    }
    
    
    class InferenceDataset(XNLIDataset):
        def __init__(self, premise_ids, hypothesis_ids):
            self.labels = None
            self.premise_ids = premise_ids
            self.hypothesis_ids = hypothesis_ids
            self.context_tokens_1 = tokenizer.encode("mnli hypothesis:")[:-1]
            self.context_tokens_2 = tokenizer.encode("premise:")[:-1]
            self.tokenizer = None
            self.max_len = 64
    
    
    collate_fn = partial(
        collate_batch, pad=model.config.decoder_start_token_id,
        decode_start_token=model.config.pad_token_id,
        max_len=128, is_classifier=False
    )
    
    premise_ids, hypothesis_ids, _ = process_file(df_test, tokenizer, batch_size=32)
    dataset = InferenceDataset(premise_ids, hypothesis_ids)
    data_loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=1, collate_fn=collate_fn)
    
    
    preds = []
    for input_batch, _ in data_loader:
        for key, val in input_batch.items():
            input_batch[key] = val.cuda()
        outputs = model(**input_batch)
        preds_local = [
            x for x in torch.argmax(outputs["logits"][:, 0, :], dim=-1).cpu().numpy()
        ]
        preds.append(preds_local)
    
    preds = np.concatenate(preds)
    # inverse_label_dict = {key: tokenizer.decode([key]) for key in np.unique(preds)}
    
    df_sub = pd.DataFrame({
        "id": df_test.id.values,
        "prediction": preds  # [inverse_label_dict[x] for x in preds]
    })
    df_sub.to_csv("submission.csv", index=False)    

In [ ]:
!python tmp.py

In [ ]:
!head submission.csv